# DCGAN in theory and practice

In [1]:
import altair as alt
import numpy as np
import tensorflow as tf

tfgan = tf.contrib.gan

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Theory" data-toc-modified-id="Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Theory</a></span><ul class="toc-item"><li><span><a href="#Enter-DCGAN" data-toc-modified-id="Enter-DCGAN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Enter DCGAN</a></span></li><li><span><a href="#Generator:-from-noise-to-insight" data-toc-modified-id="Generator:-from-noise-to-insight-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generator: from noise to insight</a></span><ul class="toc-item"><li><span><a href="#Deconvolution" data-toc-modified-id="Deconvolution-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Deconvolution</a></span></li><li><span><a href="#Batch-Normalization" data-toc-modified-id="Batch-Normalization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Batch Normalization</a></span></li><li><span><a href="#generator_fn()" data-toc-modified-id="generator_fn()-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>generator_fn()</a></span></li></ul></li><li><span><a href="#Discriminator" data-toc-modified-id="Discriminator-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Discriminator</a></span><ul class="toc-item"><li><span><a href="#discriminator_fn()" data-toc-modified-id="discriminator_fn()-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>discriminator_fn()</a></span></li></ul></li><li><span><a href="#Loss-function:-a-bridge-between-two-networks" data-toc-modified-id="Loss-function:-a-bridge-between-two-networks-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Loss function: a bridge between two networks</a></span></li></ul></li><li><span><a href="#Practice---Introduction" data-toc-modified-id="Practice---Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Practice - Introduction</a></span><ul class="toc-item"><li><span><a href="#Input-Functions" data-toc-modified-id="Input-Functions-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Input Functions</a></span><ul class="toc-item"><li><span><a href="#Loading-Images" data-toc-modified-id="Loading-Images-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Loading Images</a></span></li></ul></li><li><span><a href="#Hyperparameters" data-toc-modified-id="Hyperparameters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Hyperparameters</a></span></li><li><span><a href="#Training" data-toc-modified-id="Training-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Training</a></span></li><li><span><a href="#TensorBoard" data-toc-modified-id="TensorBoard-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>TensorBoard</a></span></li></ul></li><li><span><a href="#NOTES" data-toc-modified-id="NOTES-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>NOTES</a></span></li><li><span><a href="#Links" data-toc-modified-id="Links-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Links</a></span></li></ul></div>

## Theory

### Enter DCGAN

![DCGAN](images/dcgan.png)

If you take a look at an [impressive list of GANs](), you see that DCGAN, the architecture of our choice, is just a small drop in the ocean. While there may be sexier ones, very few offers the same package of clarity, stability, performances and computational efficiency, these are the main one reason why DCGAN is considered one of the cornerstones of this field.

Proposed in a 2015 paper by Alec Radford, Luke Metz, Soumith Chintala, the idea behind DCGAN is quite straightforward: combining a set of architectural constraints with the power of CNN yields us a robust, stable, and competitive model. Succeeding where many others failed, the authors present us with a simple model, asymmetrical architecture, 4 deconvolutional layers at the Generator with 4 layers of convolution at the Discriminator with the following constraints:

- All pooling layers are replaced with stranded convolutions (discriminator) and fractional-stride convolutions (generator).
- Batch-normalization used in both networks.
- Removal of fully connected layers (except for the discriminator output)
- `ReLU` for all Generator layers except the output, which uses `tanh`.
- `LeakyReLU` activation in the discriminator for all layers.

While there are have been very recent advancements in state of the art (i.e., CoordConv, Spectral Normalization), we felt that before venturing into the bleediest of the edges it is essential to have a firm understanding of the basic concept. We leave you (an opinionated) list of further resources you can use to get up to speed to the most exciting line of research.

### Generator: from noise to insight

![DCGAN Generator](images/dcgan_generator.png)

If you recall the theoretical explanation, the Generator is the network responsible for the data-generation, learning how to fool its discriminator allows it to produce realistic results "sampled' from the learned manifold.

The most common type of generator_inputs is pure and straightforward noise; however, more specialized GANs may require additional parameters. Since our full-demo uses a Deep Convolutional GAN (DCGAN), we don't need any other parameters.

#### Deconvolution

> When we have neural networks generate images, we often have them build them up from low resolution, high-level 
> descriptions. This allows the network to describe the rough image and then fill in the details.
>
> In order to do this, we need some way to go from a lower resolution image to a higher one. We generally do this with the deconvolution operation. 
> Roughly, deconvolution layers allow the model to use every point in the small image to “paint” a square in the larger one.

We have implemented the deconvolution layers following the findings explained [here]().

The core idea behind the paper is that boundaries and checkerboards artifacts are not due the adversarial training but are inherent to the deconvolution process, to tackle this issue, the suggested solution is an improved version of the upsampling operation, called the **different resize-convolution** whose core is a resize operation powered by either **nearest-neighbor interpolation** or **bilinear interpolation**. 

As in the paper, we are going with the ** nearest-neighbor** one.

In [2]:
def deconv2d(inputs, filters, strides=(1, 1), activation=tf.nn.relu):
    """\"Deconvolution\" layer.
    
    It uses upsampling with nearest neighbor interpolation to reduce the
    presence of checkboard artifacts.
    """
    
    input_h, input_w = inputs.shape[1].value, inputs.shape[2].value
    layer_1 = tf.image.resize_nearest_neighbor(
        inputs, (2 * input_h, 2 * input_w), name="NNUpSample2D"
    )
    # Padding before convolution is used to reduce boundary artifacts
    layer_1 = tf.pad(layer_1, [[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT")
    layer_2 = tf.layers.conv2d(
        inputs=layer_1,
        filters=filters,
        kernel_size=5,
        padding="valid",
        use_bias=False,
        activation=activation,
        strides=strides,
    )
    return layer_2

#### Batch Normalization

We do not enter into the theoretical depth of `BatchNorm` as the discussion on why or whether it works it's still open and going [NOTE 1](). What you need to know is that TensorFlow makes it very easy to implement such an operation.

```python
tf.layers.batch_normalization(input, training=is_training)
```


#### generator_fn()

In [3]:
def generator_fn(inputs, mode):
    """Generator producing images from noise.
        Args:
            noise: A single Tensor representing noise.
            mode: Either TRAIN, EVAL or PREDICT
        Returns:
            A 64x64 (None, 4096) flattened tensor whose values are
            inside the (-1, 1) range."""
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    linear = tf.layers.dense(inputs=inputs, units=1024 * 4 * 4, activation=tf.nn.relu)
    net = tf.reshape(linear, (-1, 4, 4, 1024))
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = deconv2d(net, 64)
    net = tf.layers.conv2d(
        inputs=net,
        filters=3,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=(1, 1),)
    output = tf.tanh(net)
    print(output)
    return output

### Discriminator

![DCGAN Discriminator](images/dcgan_discriminator.png)

As for the `input_fn()` we can reuse all the code we defined for the vanilla `Estimator` describing the discriminator network architecture.

#### discriminator_fn()

In [4]:
def conv2d(inputs, filters, strides=(2,2)):
    """Helper layer used to instatiate `tf.layers.conv2d` with proper arguments."""
    layer_1 = tf.layers.conv2d(
        inputs=inputs,
        filters=filters,
        kernel_size=5,
        padding="same",
        data_format="channels_last",
        use_bias=False,
        strides=strides,
    )
    layer_1 = tf.nn.leaky_relu(layer_1, alpha=0.2)

    return layer_1


In [5]:
def discriminator_fn(inputs, conditioning, mode):
    """Build the Discriminator network.
    Args:
        features: a batch of images to classify, expected input shape (None, 64, 64 , 3)
        labels: a batch of labels
        mode: the tf.estimator.ModeKey
        params: a dict of optional parameters
    Returns:
            The output (logits) of the discriminator.
    """
    
    # In every mode, define the model
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    net = conv2d(inputs, filters=128)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = conv2d(net, filters=256)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = conv2d(net, filters=512)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = conv2d(net, filters=1024)
    net = tf.layers.batch_normalization(net, training=is_training)
    net = tf.reshape(net, (-1, net.shape[1] * net.shape[2] * net.shape[3]))
    net = tf.layers.dense(net, units=1)
    out = tf.identity(net)
    
    return out

### Loss function: a bridge between two networks

There are a lot of loss functions usable in GANs'architectures from very domain-specific ones to ones that are perfect for general use cases. Since our goal here falls in the latter category, we use the so-called ** Non-Saturating Loss** which is the non-saturating variant of the **MinMax Loss** proposed by Goodfellow in the [original paper]().

As stated earlier, one of the beauties of `TFGAN` is its offerings losses ready for use, if you cannot find the loss you want, you can create your own (though this topic is outside the scope of this workshop, we leave an example of a low-level re-implementation of the `NS MinMax` in the **Beyond the Basics** section).

We use the following two losses:

```python
generator_loss_fn=tfgan.losses.minimax_generator_loss

discriminator_loss_fn=tfgan.losses.minimax_discriminator_loss
```

It is that easy.

## Practice - Introduction

Having seen the general structure of the TFGAN library, we dive right into the model architecture.

In our showcasing of the TensorFlow API, we have built an image recognition network, GANs, however, require the training of both a Generator and a Discriminator together. While this task is either verbose and possibly performance-constrained (in the case of writing it in vanilla TensorFlow) or complicated (Estimator API) thanks to GANEstimator it becomes remarkably simple.

### Input Functions

The input function for the `GANEstimator` is very much the same as the one we would define for a normal `tf.Estimator`.


>    The function should construct and return one of the following:
>
>    A tf.data.Dataset object: Outputs of Dataset object must be a tuple (features, labels) with same constraints as below.
>
>    A tuple (features, labels): Where `features` is a Tensor or a dictionary of string feature name to Tensor and >`labels` is a Tensor or a dictionary of string label name to Tensor. Model_fn consumes both features and labels. They should satisfy the expectation of model_fn from inputs.


We write our input function following the style and structure presented available on [TensorFlow Models](#1) as an example.

> ```python
def _get_train_input_fn(batch_size, noise_dims, dataset_dir=None,
                        num_threads=4):
  def train_input_fn():
    with tf.device('/cpu:0'):
      images, _, _ = data_provider.provide_data(
          'train', batch_size, dataset_dir, num_threads=num_threads)
    noise = tf.random_normal([batch_size, noise_dims])
    return noise, images
return train_input_fn
>```

The two main ideas to take from this are: 

1. structure the code in the same way, the `GANEstimator` wants **function objects** as inputs thus the need to wrap them. 
2. Return **noise** first and then the **real_data** (here called `images`) and optionally **labels**.
3. **EXP:** while often you want to work with **features (and labels)** fetched from a `tf.Dataset` [see REFERENCE](), the **noise** should always be instantiated using a simple TensorFlow node. Trying to create a noise-containing `tf.Dataset` for the `train_input_fn` is not worth the effort. NOTE: that as [we will see later](), things are different for the`_get_predict_input_fn`. 

#### Loading Images

We reuse the same `input_fn()` we defined earlier with minor changes in order to make it consistent with the aforementioned code strfucture.

In [6]:
def _get_train_images_input_fn(file_pattern, image_size=(64, 64, 3), shuffle=False,
                 batch_size=32, num_epochs=None, buffer_size=4096):
    """get_input_fn exploits the `file_pattern` to create an input_fn that reads all the content
    of the specified pattern, creating an object dataset.
    
    Args:
        file_pattern: python string, the pattern of the file to read to generate the dataset
        image_size: the new size of the read images
        shuffle: True if the order of the elements in the generated dataset shold be randomized
        batch_size: the size of the batches
        num_epochs: the number of epochs to repeat the dataset before throwing an exeption; None is unlimited
        buffer_size: how many images read before starting to generate output
    Returns:
        input_fn: the generated input_fn that returns a correctly instantiated iterator
    """
    
    def _img_string_to_tensor(image_string):
        """Decode an image as read from a `tf.decode_raw`, scales it between 0-1 and resize the
        image as specified in the parent method.
        Args:
            image_string: the raw image tensor
        Returns:
            image_resize: image in [0,1] correctly resized
        """
        
        nonlocal image_size
        
        image_decoded = tf.image.decode_jpeg(image_string, channels=image_size[-1])
        # The conversion to float automatically scales the values in [0., 1.]
        image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        image_decoded = (image_decoded_as_float - 0.5) * 2
        image_resized = tf.image.resize_images(image_decoded, size=image_size[:2])
        

        return image_resized

    def _path_to_img(path):
        """Given the path of an image, returns the pair (image, label)
        where image is the corretly resized image, and label is the label associated with it.
        Args:
            path: the path of the image to read
        Returns:
            (image_resized, label): the image, label pair associated the path
        """
        
        # Get the parent folder of this file to get its class
        # Associate the label 0 to dogs and 1 to cats
        label = tf.cond(
                    tf.equal(tf.string_split([path], delimiter='/').values[-2], "dogs"),
                    lambda: 0, lambda: 1)

        image_string = tf.read_file(path) # read image and process it
        image_resized = _img_string_to_tensor(image_string)

        return image_resized, label
    
    def _input_fn():
        """The input function that builds the `tf.data.Dataset` object and instantiate
        the iterator correctly ready to be use.
        Returns:
            the iterator associated to the built Dataset object.
        """
        
        # Use the static method `list_files` that builds a dataset of all
        # files matching this pattern.
        dataset_path = tf.data.Dataset.list_files(file_pattern)

        if shuffle:
            dataset_path = dataset_path.apply(tf.contrib.data.shuffle_and_repeat(buffer_size, num_epochs))
        else:
            dataset_path = dataset_path.repeat(num_epochs)

        # The map function maps the path to the pair (image, label)
        dataset = dataset_path.map(_path_to_img)
        dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
        dataset = dataset.prefetch(buffer_size)
        
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()

    return _input_fn()[0]

We define the `_get_train_input_fn()` which return a Python Callable that we instantiate and pass to `GANEstimator.train()` later on.

In [7]:
def _get_train_input_fn(file_pattern, batch_size, num_epochs, noise_dims=100, **kwargs):
    def train_input_fn():
        real_data = _get_train_images_input_fn(
                 file_pattern,
                 batch_size=batch_size, 
                 num_epochs=num_epochs)
        noise = tf.random_normal([batch_size, noise_dims], name="train_noise")
        real_data.set_shape((batch_size,) + tuple(real_data.shape[1:]))
        print(noise, real_data)
        return noise, real_data
    return train_input_fn

### Hyperparameters

Personally I like to keep my hyperparameters in a **TOML** file, loading them before the train.

Here we simply load them in a `Dict` and use `**` to unpack them.

In [8]:
def load_hyperparameters():
    hp = {
        "model_dir": "../logs/",
        "file_pattern": "../2. GANs in Tensorflow/dogscats/train/**/*.jpg",
        "batch_size": 128,
        "num_epochs":1000,
        "noise_dims": 100
    }
    return hp

### Training

In [9]:
def dcgan():
    
    # Hyperparameters
    hyperparameters = load_hyperparameters()
    
    # Run Configuration (it has other arguments)
    run_config = tf.estimator.RunConfig(
        model_dir=hyperparameters.get("model_dir"), save_summary_steps=50, save_checkpoints_steps=500)
    
    # Instatiate the GANEstimator object
    gan_estimator = tfgan.estimator.GANEstimator(
        config=run_config,
        generator_fn=generator_fn,
        discriminator_fn=discriminator_fn,
        generator_loss_fn=tfgan.losses.modified_generator_loss,
        discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
        generator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        discriminator_optimizer=tf.train.AdamOptimizer(0.0002, 0.5),
        add_summaries=tfgan.estimator.SummaryType.IMAGES
    )
    
    # Instatiate the train_input_fn
    # The model will train until it exhausts the Dataset which is repeated EPOCH times
    train_input_fn = _get_train_input_fn(**hyperparameters)
    trained_model = gan_estimator.train(train_input_fn, max_steps=None)
    return trained_model

### TensorBoard

In order to track our training we need to launch a **TensorBoard** session pointing to the folder (`model_dir`) containing the logs generated by our training.

```bash
tensorboard --logdir=PATH_TO_YOUR_MODEL_DIR
```

In [10]:
trained_model = dcgan()

INFO:tensorflow:Using config: {'_model_dir': '../logs/', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0e7b331be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Tensor("train_noise:0", shape=(128, 100), dtype=float32, device=/device:CPU:0) Tensor("IteratorGetNext:0", shape=(128, 64, 64, 3), dtype=float32, device=/device:CPU:0)
INFO:tensorflow:Calling model_fn.
Tensor("Generator/Tanh:0", shape=(128, 64, 64, 3), dtype=float32)
INFO:tensorflow:Summary name Generator/dense/kernel:0 is illegal; using Generator/dense/kernel_0 in

INFO:tensorflow:global_step/sec: 1.7648
INFO:tensorflow:loss = 4.3675613, step = 33801 (56.663 sec)
INFO:tensorflow:global_step/sec: 1.76412
INFO:tensorflow:loss = 6.4518547, step = 33901 (56.686 sec)
INFO:tensorflow:Saving checkpoints for 34000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72527
INFO:tensorflow:loss = 10.858253, step = 34001 (57.962 sec)
INFO:tensorflow:global_step/sec: 1.765
INFO:tensorflow:loss = 6.28068, step = 34101 (56.657 sec)
INFO:tensorflow:global_step/sec: 1.76348
INFO:tensorflow:loss = 6.25112, step = 34201 (56.706 sec)
INFO:tensorflow:global_step/sec: 1.76372
INFO:tensorflow:loss = 7.2974973, step = 34301 (56.698 sec)
INFO:tensorflow:global_step/sec: 1.76371
INFO:tensorflow:loss = 4.905837, step = 34401 (56.698 sec)
INFO:tensorflow:Saving checkpoints for 34500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72012
INFO:tensorflow:loss = 6.519368, step = 34501 (58.136 sec)
INFO:tensorflow:global_step/sec: 1.7626
INFO:tensorflow:loss 

INFO:tensorflow:Saving checkpoints for 41000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71935
INFO:tensorflow:loss = 6.1687727, step = 41001 (58.161 sec)
INFO:tensorflow:global_step/sec: 1.76439
INFO:tensorflow:loss = 7.374987, step = 41101 (56.677 sec)
INFO:tensorflow:global_step/sec: 1.76357
INFO:tensorflow:loss = 6.8004813, step = 41201 (56.703 sec)
INFO:tensorflow:global_step/sec: 1.7628
INFO:tensorflow:loss = 6.4818535, step = 41301 (56.728 sec)
INFO:tensorflow:global_step/sec: 1.76341
INFO:tensorflow:loss = 7.7191343, step = 41401 (56.709 sec)
INFO:tensorflow:Saving checkpoints for 41500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72063
INFO:tensorflow:loss = 5.2683682, step = 41501 (58.118 sec)
INFO:tensorflow:global_step/sec: 1.76397
INFO:tensorflow:loss = 7.5246572, step = 41601 (56.690 sec)
INFO:tensorflow:global_step/sec: 1.76328
INFO:tensorflow:loss = 6.5528636, step = 41701 (56.713 sec)
INFO:tensorflow:global_step/sec: 1.76141
INFO:tensorfl

INFO:tensorflow:loss = 5.4918003, step = 48101 (56.681 sec)
INFO:tensorflow:global_step/sec: 1.7499
INFO:tensorflow:loss = 8.116854, step = 48201 (57.146 sec)
INFO:tensorflow:global_step/sec: 1.76271
INFO:tensorflow:loss = 5.160519, step = 48301 (56.731 sec)
INFO:tensorflow:global_step/sec: 1.76396
INFO:tensorflow:loss = 5.94569, step = 48401 (56.691 sec)
INFO:tensorflow:Saving checkpoints for 48500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71883
INFO:tensorflow:loss = 7.1108923, step = 48501 (58.179 sec)
INFO:tensorflow:global_step/sec: 1.76277
INFO:tensorflow:loss = 6.105492, step = 48601 (56.729 sec)
INFO:tensorflow:global_step/sec: 1.76317
INFO:tensorflow:loss = 7.539501, step = 48701 (56.716 sec)
INFO:tensorflow:global_step/sec: 1.76355
INFO:tensorflow:loss = 7.745761, step = 48801 (56.704 sec)
INFO:tensorflow:global_step/sec: 1.76385
INFO:tensorflow:loss = 7.1053495, step = 48901 (56.694 sec)
INFO:tensorflow:Saving checkpoints for 49000 into ../logs/model.ckpt.


INFO:tensorflow:global_step/sec: 1.76341
INFO:tensorflow:loss = 6.2219424, step = 55401 (56.708 sec)
INFO:tensorflow:Saving checkpoints for 55500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71329
INFO:tensorflow:loss = 7.413375, step = 55501 (58.381 sec)
INFO:tensorflow:global_step/sec: 1.76204
INFO:tensorflow:loss = 6.752772, step = 55601 (56.739 sec)
INFO:tensorflow:global_step/sec: 1.76308
INFO:tensorflow:loss = 6.829056, step = 55701 (56.718 sec)
INFO:tensorflow:global_step/sec: 1.76437
INFO:tensorflow:loss = 7.183496, step = 55801 (56.678 sec)
INFO:tensorflow:global_step/sec: 1.76368
INFO:tensorflow:loss = 5.5365353, step = 55901 (56.700 sec)
INFO:tensorflow:Saving checkpoints for 56000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72011
INFO:tensorflow:loss = 7.562919, step = 56001 (58.135 sec)
INFO:tensorflow:global_step/sec: 1.7649
INFO:tensorflow:loss = 5.9322166, step = 56101 (56.661 sec)
INFO:tensorflow:global_step/sec: 1.76375
INFO:tensorflow:l

...

INFO:tensorflow:loss = 8.090485, step = 91301 (56.672 sec)
INFO:tensorflow:global_step/sec: 1.76402
INFO:tensorflow:loss = 8.554741, step = 91401 (56.689 sec)
INFO:tensorflow:Saving checkpoints for 91500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71915
INFO:tensorflow:loss = 7.992671, step = 91501 (58.168 sec)
INFO:tensorflow:global_step/sec: 1.7648
INFO:tensorflow:loss = 7.99718, step = 91601 (56.664 sec)
INFO:tensorflow:global_step/sec: 1.76524
INFO:tensorflow:loss = 9.059976, step = 91701 (56.649 sec)
INFO:tensorflow:global_step/sec: 1.76482
INFO:tensorflow:loss = 7.1232157, step = 91801 (56.663 sec)
INFO:tensorflow:global_step/sec: 1.76425
INFO:tensorflow:loss = 9.129687, step = 91901 (56.681 sec)
INFO:tensorflow:Saving checkpoints for 92000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71977
INFO:tensorflow:loss = 7.8013926, step = 92001 (58.149 sec)
INFO:tensorflow:global_step/sec: 1.76516
INFO:tensorflow:loss = 8.0294485, step = 92101 (56.650 sec)


INFO:tensorflow:global_step/sec: 1.72036
INFO:tensorflow:loss = 8.514343, step = 98501 (58.127 sec)
INFO:tensorflow:global_step/sec: 1.76421
INFO:tensorflow:loss = 8.103905, step = 98601 (56.683 sec)
INFO:tensorflow:global_step/sec: 1.76465
INFO:tensorflow:loss = 7.6559815, step = 98701 (56.668 sec)
INFO:tensorflow:global_step/sec: 1.76396
INFO:tensorflow:loss = 7.685789, step = 98801 (56.691 sec)
INFO:tensorflow:global_step/sec: 1.76546
INFO:tensorflow:loss = 6.7633696, step = 98901 (56.642 sec)
INFO:tensorflow:Saving checkpoints for 99000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72157
INFO:tensorflow:loss = 7.7743516, step = 99001 (58.086 sec)
INFO:tensorflow:global_step/sec: 1.76486
INFO:tensorflow:loss = 8.589191, step = 99101 (56.662 sec)
INFO:tensorflow:global_step/sec: 1.76383
INFO:tensorflow:loss = 8.548388, step = 99201 (56.695 sec)
INFO:tensorflow:global_step/sec: 1.76635
INFO:tensorflow:loss = 6.1918964, step = 99301 (56.614 sec)
INFO:tensorflow:global_ste

INFO:tensorflow:loss = 8.945088, step = 105601 (56.654 sec)
INFO:tensorflow:global_step/sec: 1.76339
INFO:tensorflow:loss = 8.479673, step = 105701 (56.709 sec)
INFO:tensorflow:global_step/sec: 1.76555
INFO:tensorflow:loss = 10.0686865, step = 105801 (56.640 sec)
INFO:tensorflow:global_step/sec: 1.76284
INFO:tensorflow:loss = 9.390171, step = 105901 (56.727 sec)
INFO:tensorflow:Saving checkpoints for 106000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72292
INFO:tensorflow:loss = 6.8725333, step = 106001 (58.041 sec)
INFO:tensorflow:global_step/sec: 1.76252
INFO:tensorflow:loss = 9.666389, step = 106101 (56.737 sec)
INFO:tensorflow:global_step/sec: 1.76383
INFO:tensorflow:loss = 8.857022, step = 106201 (56.695 sec)
INFO:tensorflow:global_step/sec: 1.76481
INFO:tensorflow:loss = 8.598107, step = 106301 (56.664 sec)
INFO:tensorflow:global_step/sec: 1.76324
INFO:tensorflow:loss = 9.170214, step = 106401 (56.713 sec)
INFO:tensorflow:Saving checkpoints for 106500 into ../logs

INFO:tensorflow:global_step/sec: 1.76157
INFO:tensorflow:loss = 9.295657, step = 112801 (56.768 sec)
INFO:tensorflow:global_step/sec: 1.76313
INFO:tensorflow:loss = 7.5056024, step = 112901 (56.717 sec)
INFO:tensorflow:Saving checkpoints for 113000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.7225
INFO:tensorflow:loss = 9.241611, step = 113001 (58.055 sec)
INFO:tensorflow:global_step/sec: 1.76377
INFO:tensorflow:loss = 9.544763, step = 113101 (56.697 sec)
INFO:tensorflow:global_step/sec: 1.76351
INFO:tensorflow:loss = 9.730476, step = 113201 (56.705 sec)
INFO:tensorflow:global_step/sec: 1.76304
INFO:tensorflow:loss = 8.655949, step = 113301 (56.720 sec)
INFO:tensorflow:global_step/sec: 1.76336
INFO:tensorflow:loss = 8.86189, step = 113401 (56.710 sec)
INFO:tensorflow:Saving checkpoints for 113500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71991
INFO:tensorflow:loss = 6.776365, step = 113501 (58.142 sec)
INFO:tensorflow:global_step/sec: 1.76403
INFO:tenso

INFO:tensorflow:loss = 10.227643, step = 119901 (56.706 sec)
INFO:tensorflow:Saving checkpoints for 120000 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.71976
INFO:tensorflow:loss = 10.63669, step = 120001 (58.148 sec)
INFO:tensorflow:global_step/sec: 1.7657
INFO:tensorflow:loss = 7.913427, step = 120101 (56.635 sec)
INFO:tensorflow:global_step/sec: 1.76453
INFO:tensorflow:loss = 6.285139, step = 120201 (56.672 sec)
INFO:tensorflow:global_step/sec: 1.76287
INFO:tensorflow:loss = 6.956264, step = 120301 (56.726 sec)
INFO:tensorflow:global_step/sec: 1.76435
INFO:tensorflow:loss = 6.093244, step = 120401 (56.678 sec)
INFO:tensorflow:Saving checkpoints for 120500 into ../logs/model.ckpt.
INFO:tensorflow:global_step/sec: 1.72399
INFO:tensorflow:loss = 6.3243, step = 120501 (58.005 sec)
INFO:tensorflow:global_step/sec: 1.76394
INFO:tensorflow:loss = 7.4976172, step = 120601 (56.692 sec)
INFO:tensorflow:global_step/sec: 1.76385
INFO:tensorflow:loss = 7.4489207, step = 120701 (56

## NOTES

[1]: [The GAN Landscape: Losses, Architectures, Regularization, and Normalization](https://arxiv.org/abs/1807.04720v1), which in our opinion is one of the most thorough scientific studies on GANs out there, suggests that `Spectral Normalization` is the real big deal and that `BatchNorm` actually hurt performance if applied to the Discriminator. We still decided to go with the classic formulation of DCGAN as not to overtax you with theoretical discussions. We leave the implementation of the `Spectral Normalization`(and SNGAN) to you as an exercise. On the theoretical treating of `BatchNorm` we recommend a back to back reading of [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167), [Understanding Batch Normalization](https://arxiv.org/abs/1806.02375v1), [How Does Batch Normalization Help Optimization? (No, It Is Not About Internal Covariate Shift)](https://arxiv.org/abs/1805.11604) 

## Links

Generative Adversarial Networks : https://arxiv.org/abs/1406.2661

really-awesome-gan : https://github.com/nightrome/really-awesome-gan

DCGAN : https://arxiv.org/abs/1511.06434

Deconvolution and Checkerboard Artifacts : https://distill.pub/2016/deconv-checkerboard/

TFGAN MNIST GAN Example : https://github.com/tensorflow/models/tree/master/research/gan/mnist_estimator

